# CnA Final Capstone Project Missing value Imputation

In this notebook (this is the 2nd one), I mainly do the imputation of missing values. There were quite a few missing values and several wrong entries but I ended up not exploring this aspect for this hackathon although I do mention some instances in this notebook. There is a lot of scope in these particular steps and is worth exploring

This notebook by **@debarshichanda** covers several missing value imputation techniques: https://www.kaggle.com/code/debarshichanda/handling-missing-values/notebook

## Importing Libraries

In [1]:
#Basic Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
import warnings
warnings.filterwarnings('ignore')

#Text Preprocessing
import re

# Iterative Imputation
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from xgboost import XGBRegressor

In [2]:
# Loading in the Datasets
df = pd.read_csv("../input/cna-5folds/train_folds.csv")
df_test = pd.read_csv("../input/sa2022/test.csv")
sample_submission = pd.read_csv("../input/sa2022/Sample__submission.csv")

In [3]:
# Converting Offer % from string to float to make it a useable target
# Target choice between Offer % and price1, it did seem like offer% might be a good target considering it is bounded
# Also point to note for those who did use Offer %, some of the values in this feature were wrong, or some in actprice1 were wrong
def remove_punct(text):
    text = re.sub('%','',text)
    return text
df['Offer %'] = df['Offer %'].apply(remove_punct).astype('float')

In [4]:
# According to my understanding (which maybe wrong), the norating1 column should have been (and is for several other rows) the sum of the number of stars
df[df['norating1']!=df['star_5f']+df['star_4f']+df['star_3f']+df['star_2f']+df['star_1f']]

,id,title,Rating,maincateg,platform,price1,actprice1,Offer %,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1,kfold
3,495,Men Black Sports Sandal,4.2,Men,Flipkart,518,724,15.85,46413.0,6229.0,1045.0,12416.0,5352.0,701,4595,1,0
6,3236,Women Navy Heels Sandal,4.0,Women,Flipkart,499,999,50.05,249.0,31.0,130.0,52.0,27.0,9,28,1,3
7,14633,Women Blue Wedges Sandal,3.9,Women,Flipkart,209,1299,83.91,121.0,17.0,63.0,25.0,14.0,9,12,0,3
8,1872,Sneakers For Women (Pink),4.2,Women,Flipkart,549,999,45.05,191.0,30.0,120.0,25.0,31.0,7,12,1,0
10,12634,TEMPO Men's Combo Pack of 3 Synthetic Loafers ...,3.3,Men,Amazon,999,999,0.00,222.0,147.0,26.0,4.0,18.0,15,37,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15716,11613,Bacca Bucci Men's Sneaker,3.6,Men,Amazon,999,999,0.00,686.0,296.0,17.0,7.0,14.0,23,38,1,3
15722,239,"Fancy-01 Women's Flyknit Socks Sneakers,Ultra-...",4.1,Women,Flipkart,668,1615,49.37,10283.0,1347.0,2880.0,1769.0,1350.0,3,871,1,2
15723,14422,Bellies For Women (Black),4.0,Women,Flipkart,209,499,58.12,219.0,21.0,122.0,39.0,16.0,15,24,0,1
15727,6733,Women Navy Flats Sandal,3.7,Women,Flipkart,329,499,34.07,NaN,18.0,NaN,NaN,20.0,10,15,1,4


In [5]:
# These entries are obviously incorrect and there are several such errors
df[(df['star_5f']==0)&(df['star_4f']==0)&(df['star_3f']==0)&(df['star_2f']==0)&(df['star_1f']==0)]

,id,title,Rating,maincateg,platform,price1,actprice1,Offer %,norating1,noreviews1,star_5f,star_4f,star_3f,star_2f,star_1f,fulfilled1,kfold
1197,1099,DOCTOR EXTRA SOFT Ortho Care Diabetic Orthopae...,4.0,Men,Flipkart,499,799,37.55,3046.0,425.0,0.0,0.0,0.0,0,0,1,0
1413,12126,Jutis For Men (Tan),4.3,Men,Flipkart,1530,1699,9.95,22.0,5.0,0.0,0.0,0.0,0,0,0,1
1578,16727,"Cosko Sports Shoes,Running Shoes,Walking Shoes...",4.1,Men,Flipkart,699,999,30.03,122519.0,17774.0,0.0,0.0,0.0,0,0,1,1
1783,1108,Women Multicolor Wedges Sandal,4.1,Women,Flipkart,845,1690,50.00,53.0,10.0,0.0,0.0,0.0,0,0,1,0
1997,12141,Ladies New casuals Boots For Women (Black),3.4,Women,Flipkart,899,999,10.01,8.0,0.0,0.0,0.0,0.0,0,0,0,4
2084,10286,Jutis For Men (Brown),4.0,Men,Flipkart,498,499,0.20,441.0,40.0,0.0,0.0,0.0,0,0,0,0
2368,1084,Lace Up For Men (Maroon),4.4,Men,Flipkart,2195,3990,44.99,27.0,5.0,0.0,0.0,0.0,0,0,1,4
2487,10356,Women Pink Flats Sandal,4.0,Women,Flipkart,699,999,30.03,116.0,16.0,0.0,0.0,0.0,0,0,0,1
3449,20022,Corporate Casuals For Men (Black),4.0,Men,Flipkart,421,999,57.86,2224.0,330.0,0.0,0.0,0.0,0,0,1,1
3726,12745,Modern Runner Running Shoes For Men (Black),4.1,Men,Flipkart,1799,2999,40.01,9.0,0.0,0.0,0.0,0.0,0,0,1,4


In [6]:
# Just dropped this row because it had many wrong inputs, there were several such rows but I ended up not exploring this preprocessinng step
df.drop(df[df['id']==9873].index,inplace = True)

In [7]:
# Seeing the shapes to check the column and row count is as expected
print(df.shape,df_test.shape)

(15729, 17) (5244, 14)


## Main Imputation step

In [8]:
df.isnull().sum()

id              0
title           0
Rating          0
maincateg     526
platform        0
price1          0
actprice1       0
Offer %         0
norating1     678
noreviews1    578
star_5f       588
star_4f       539
star_3f       231
star_2f         0
star_1f         0
fulfilled1      0
kfold           0
dtype: int64

In [9]:
df_test.isnull().sum()

id              0
title           0
Rating        203
maincateg      67
platform        0
actprice1       0
norating1       0
noreviews1      0
star_5f        68
star_4f         0
star_3f         0
star_2f         0
star_1f       186
fulfilled1      0
dtype: int64

In [10]:
a = df['maincateg'].isnull().sum()
b = df_test['maincateg'].isnull().sum()
print(f'Null values in maincateg (before imputation) in df: {a} and in df_test: {b}')

Null values in maincateg (before imputation) in df: 526 and in df_test: 67


In [11]:
# Main imputation step
# Here I just deal with the missing values in maincateg which I try to fill as much as I can using the title column 
# and the fact that several title columns have Men or Women specified in some form. Here I didnt spend much time on the rest of the null value columns
df.loc[(df['title'].str.contains('Women')) & (df['maincateg'].isnull() == True),'maincateg'] = df.loc[(df['title'].str.contains('Women')) & (df['maincateg'].isnull() == True),'maincateg'].fillna(value = 'Women')
df.loc[(df['title'].str.contains('Men')) & (df['maincateg'].isnull() == True),'maincateg'] = df.loc[(df['title'].str.contains('Men')) & (df['maincateg'].isnull() == True),'maincateg'].fillna(value = 'Men')
df.loc[(df['title'].str.contains('men')) & (df['maincateg'].isnull() == True),'maincateg'] = df.loc[(df['title'].str.contains('men')) & (df['maincateg'].isnull() == True),'maincateg'].fillna(value = 'Men')
df.loc[(df['title'].str.contains('Boy')) & (df['maincateg'].isnull() == True),'maincateg'] = df.loc[(df['title'].str.contains('Boy')) & (df['maincateg'].isnull() == True),'maincateg'].fillna(value = 'Men')
df.loc[(df['title'].str.contains('WOMEN')) & (df['maincateg'].isnull() == True),'maincateg'] = df.loc[(df['title'].str.contains('WOMEN')) & (df['maincateg'].isnull() == True),'maincateg'].fillna(value = 'Women')
df.loc[(df['title'].str.contains('Pink')) & (df['maincateg'].isnull() == True),'maincateg'] = df.loc[(df['title'].str.contains('Pink')) & (df['maincateg'].isnull() == True),'maincateg'].fillna(value = 'Women')
df.loc[(df['title'].str.contains('Ketava')) & (df['maincateg'].isnull() == True),'maincateg'] = df.loc[(df['title'].str.contains('Ketava')) & (df['maincateg'].isnull() == True),'maincateg'].fillna(value = 'Men')
df.loc[(df['title'].str.contains('WF_HF')) & (df['maincateg'].isnull() == True),'maincateg'] = df.loc[(df['title'].str.contains('WF_HF')) & (df['maincateg'].isnull() == True),'maincateg'].fillna(value = 'Women')

df_test.loc[(df_test['title'].str.contains('Women')) & (df_test['maincateg'].isnull() == True),'maincateg'] = df_test.loc[(df_test['title'].str.contains('Women')) & (df_test['maincateg'].isnull() == True),'maincateg'].fillna(value = 'Women')
df_test.loc[(df_test['title'].str.contains('Men')) & (df_test['maincateg'].isnull() == True),'maincateg'] = df_test.loc[(df_test['title'].str.contains('Men')) & (df_test['maincateg'].isnull() == True),'maincateg'].fillna(value = 'Men')
df_test.loc[(df_test['title'].str.contains('men')) & (df_test['maincateg'].isnull() == True),'maincateg'] = df_test.loc[(df_test['title'].str.contains('men')) & (df_test['maincateg'].isnull() == True),'maincateg'].fillna(value = 'Men')
df_test.loc[(df_test['title'].str.contains('Boy')) & (df_test['maincateg'].isnull() == True),'maincateg'] = df_test.loc[(df_test['title'].str.contains('Boy')) & (df_test['maincateg'].isnull() == True),'maincateg'].fillna(value = 'Men')
df_test.loc[(df_test['title'].str.contains('WOMEN')) & (df_test['maincateg'].isnull() == True),'maincateg'] = df_test.loc[(df_test['title'].str.contains('WOMEN')) & (df_test['maincateg'].isnull() == True),'maincateg'].fillna(value = 'Women')
df_test.loc[(df_test['title'].str.contains('Pink')) & (df_test['maincateg'].isnull() == True),'maincateg'] = df_test.loc[(df_test['title'].str.contains('Pink')) & (df_test['maincateg'].isnull() == True),'maincateg'].fillna(value = 'Women')
df_test.loc[(df_test['title'].str.contains('Wink')) & (df_test['maincateg'].isnull() == True),'maincateg'] = df_test.loc[(df_test['title'].str.contains('Wink')) & (df_test['maincateg'].isnull() == True),'maincateg'].fillna(value = 'Men')
df_test.loc[(df_test['maincateg'].isnull() == True),'star_1f'] = df_test.loc[(df_test['maincateg'].isnull() == True),'norating1']-df_test.loc[(df_test['maincateg'].isnull() == True),'star_5f']-df_test.loc[(df_test['maincateg'].isnull() == True),'star_4f']-df_test.loc[(df_test['maincateg'].isnull() == True),'star_3f']-df_test.loc[(df_test['maincateg'].isnull() == True),'star_2f']

# Did some manual entry correction but was not motivated enough to spend more time on it :(
df.loc[df['id'] == 5099,'star_5f'] = 130
df.loc[df['id'] == 2159,'star_4f'] = 29
df.loc[df['id'] == 9850,'star_5f'] = 210
df.loc[df['id'] == 11602,'star_4f'] = 28

In [12]:
a = df['maincateg'].isnull().sum()
b = df_test['maincateg'].isnull().sum()
print(f'Null values in maincateg (after imputation) in df: {a} and in df_test: {b}')

Null values in maincateg (after imputation) in df: 53 and in df_test: 6


In [13]:
df = df.drop(['title'],axis='columns')
df_test = df_test.drop(['title'],axis='columns')

In [14]:
# Label Encoding using map function for the binary categorical variables
df['maincateg'] = df['maincateg'].map({'Women':0,'Men':1})
df['platform'] = df['platform'].map({'Amazon':0,'Flipkart':1})
df_test['maincateg'] = df_test['maincateg'].map({'Women':0,'Men':1})
df_test['platform'] = df_test['platform'].map({'Amazon':0,'Flipkart':1})

In [15]:
# The Main Iterative Imputation Step
imputer = IterativeImputer(estimator=XGBRegressor(n_estimators=2000,tree_method='gpu_hist', predictor="gpu_predictor"),max_iter=20,verbose = 2)
df = pd.DataFrame(imputer.fit_transform(df),columns = df.columns)
imputer = IterativeImputer(estimator=XGBRegressor(n_estimators=2000,tree_method='gpu_hist', predictor="gpu_predictor"),max_iter=20,verbose = 2)
df_test = pd.DataFrame(imputer.fit_transform(df_test),columns = df_test.columns)

[IterativeImputer] Completing matrix with shape (15729, 16)
[IterativeImputer] Ending imputation round 1/20, elapsed time 100.84
[IterativeImputer] Change: 388743.7066326436, scaled tolerance: 289.973 
[IterativeImputer] Ending imputation round 2/20, elapsed time 196.27
[IterativeImputer] Change: 82576.171875, scaled tolerance: 289.973 
[IterativeImputer] Ending imputation round 3/20, elapsed time 292.44
[IterativeImputer] Change: 9607.146484375, scaled tolerance: 289.973 
[IterativeImputer] Ending imputation round 4/20, elapsed time 388.78
[IterativeImputer] Change: 10527.84912109375, scaled tolerance: 289.973 
[IterativeImputer] Ending imputation round 5/20, elapsed time 484.92
[IterativeImputer] Change: 8244.74267578125, scaled tolerance: 289.973 
[IterativeImputer] Ending imputation round 6/20, elapsed time 580.99
[IterativeImputer] Change: 2423.6689453125, scaled tolerance: 289.973 
[IterativeImputer] Ending imputation round 7/20, elapsed time 678.24
[IterativeImputer] Change: 282

In [16]:
# Converting the train and test datasets to csv files
df.to_csv('train.csv',index = False)
df_test.to_csv('test.csv',index = False)